In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# !pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

In [ ]:
# !pip install --upgrade -q ultralytics

In [ ]:
import torch
torch.backends.cuda.matmul.allow_tf32 = True  # Giúp tăng hiệu suất trên GPU
torch.backends.cudnn.benchmark = True  # Cho phép CUDNN chọn thuật toán tối ưu nhất
torch.use_deterministic_algorithms(True, warn_only=False)

In [ ]:
!yolo detect train data=/kaggle/input/brackish/data.yaml \
model= /kaggle/input/yolo12/pytorch/underwater/2/best.pt \
epochs=50 device=0,1 batch=128 workers=4 optimizer=AdamW dropout=0.2\
lr0=1e-4, lrf=2e-4

## Evaluate fine-tuned YOLOv12 model

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!ls {HOME}/runs/detect/train2/

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train2/confusion_matrix.png', width=1000)

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train2/results.png', width=1000)

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"/kaggle/input/brackish/test/images",
    annotations_directory_path=f"/kaggle/input/brackish/test/labels",
    data_yaml_path=f"/kaggle/input/brackish/data.yaml"
)

ds.classes

In [ ]:
from supervision.metrics import MeanAveragePrecision
from yolo12 import YOLO
model = YOLO(f'/{HOME}/runs/detect/train2/weights/best.pt')

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()

In [ ]:
print("mAP 50:95", map.map50_95)
print("mAP 50", map.map50)
print("mAP 75", map.map75)

In [ ]:
map.plot()

## Run inference with fine-tuned YOLOv12 model

In [ ]:
import supervision as sv

model = YOLO(f'/{HOME}/runs/detect/train2/weights/best.pt')

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"/kaggle/input/brackish/test/images",
    annotations_directory_path=f"/kaggle/input/brackish/test/labels",
    data_yaml_path=f"/kaggle/input/brackish/data.yaml"
)

In [ ]:
import random

i = random.randint(0, len(ds))

image_path, image, target = ds[1]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

###### batch

In [ ]:
!zip ./output.zip -r /kaggle/working/runs